# Проект 

# Описание проекта



# Цели исследования

## Решаемые задачи
1. Посчитать, сколько книг вышло после 1 января 2000 года;
1. Для каждой книги посчитать количество обзоров и среднюю оценку;
1. Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так мы исключим из анализа брошюры;
1. Определить автора с самой высокой средней оценкой книг, учитывая только книги с 50 и более оценками;
1. Посчитать среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

# Описание данных 

### `books`  
Содержит данные о книгах:

- `book_id` — идентификатор книги
- `author_id` — идентификатор автора
- `title` — название книги
- `num_pages` — количество страниц
- `publication_date` — дата публикации книги
- `publisher_id` — идентификатор издателя

### `authors`  
Содержит данные об авторах:

- `author_id` — идентификатор автора
- `author` — имя автора

### `publishers`  
Содержит данные об издательствах:

- `publisher_id` — идентификатор издательства
- `publisher` — название издательства

### `ratings`  
Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки
- `book_id` — идентификатор книги
- `username` — имя пользователя, оставившего оценку
- `rating` — оценка книги

### `reviews`  
Содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора
- `book_id` — идентификатор книги
- `username` — имя пользователя, написавшего обзор
- `text` — текст обзора

# Исследование 

## Подключение к базе данных

In [8]:
# импортируем библиотеки
import pandas as pd
import sqlalchemy as sa
# устанавливаем параметры
db_config = {
'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db' # название базы данных
}
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)
# сохраняем коннектор
engine = sa.create_engine(connection_string, connect_args={'sslmode':'require'})


Изучим содержание таблицы books

In [9]:
query = """
SELECT *
FROM books
LIMIT 5;
"""
response = pd.io.sql.read_sql(query, con = engine)
response

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


## Количество книг, вышедших после 1 января 2000 года 
Отберём книги, вышедшие после 1 января 2000 года, и посчитаем их количество.

In [10]:
query = """
SELECT count(book_id) 
FROM books 
WHERE publication_date > '2000-01-01';
"""

response = pd.io.sql.read_sql(query, con = engine)
response

,count
0,819


## Вывод 
После 1 января 2020 года вышло 819 книг. 

## Количество обзоров и средняя оценка для каждой книги 
Для запроса возьмём таблицы books, reviews и ratings 

In [11]:
query = """
SELECT title,
       COUNT(DISTINCT review_id) AS review_cnt,
       ROUND(AVG(rating), 2) AS rating_avg
FROM books AS b 
     LEFT JOIN ratings AS r 
     ON b.book_id = r.book_id
     LEFT JOIN reviews AS rv 
     ON b.book_id = rv.book_id
GROUP BY b.book_id
ORDER BY rating_avg DESC, 
         review_cnt DESC

"""
response = pd.io.sql.read_sql(query, con = engine)
response

,title,review_cnt,rating_avg
0,A Dirty Job (Grim Reaper #1),4,5.00
1,School's Out—Forever (Maximum Ride #2),3,5.00
2,Moneyball: The Art of Winning an Unfair Game,3,5.00
3,The Big Bad Wolf (Alex Cross #9),2,5.00
4,Welcome to Temptation (Dempseys #1),2,5.00
...,...,...,...
995,The World Is Flat: A Brief History of the Twen...,3,2.25
996,Drowning Ruth,3,2.00
997,His Excellency: George Washington,2,2.00
998,Junky,2,2.00


## Вывод 
Посчитали количество обзоров и средний рейтинг для каждой книги. Вывели информацию по количеству обзоров и рейтингу - по убыванию. 

## Издательство, выпустившее наибольшее количество книг толщиной более 50 страниц 

In [12]:
query = """
SELECT 
    p.publisher,
    COUNT(b.book_id) AS books_count
FROM 
    publishers p
JOIN 
    books b ON p.publisher_id = b.publisher_id
WHERE 
    b.num_pages > 50
GROUP BY 
    p.publisher_id, p.publisher
ORDER BY 
    books_count DESC
LIMIT 1;

"""
response = pd.io.sql.read_sql(query, con = engine)
response

,publisher,books_count
0,Penguin Books,42


## Вывод 
Больше всего книг толщиной больше 50 страниц выпустило издательство Penguin Books - 42 книги. 

##  Автор с самой высокой средней оценкой книг 
Будем учитывать только книги, собравшие больше 50 оценок 

In [13]:
query = """
WITH rated_books AS (
    SELECT 
        b.book_id,
        b.author_id,
        b.title,
        COUNT(r.rating_id) AS ratings_count,
        AVG(r.rating) AS avg_rating
    FROM 
        books b
    JOIN 
        ratings r ON b.book_id = r.book_id
    GROUP BY 
        b.book_id, b.author_id, b.title
    HAVING 
        COUNT(r.rating_id) > 50
)
SELECT 
    a.author,
    ROUND(AVG(rb.avg_rating), 2) AS avg_author_rating,
    COUNT(rb.book_id) AS qualified_books_count
FROM 
    rated_books rb
JOIN 
    authors a ON rb.author_id = a.author_id
GROUP BY 
    a.author_id, a.author
ORDER BY 
    avg_author_rating DESC
LIMIT 1;
"""

response = pd.io.sql.read_sql(query, con = engine)
response

,author,avg_author_rating,qualified_books_count
0,J.K. Rowling/Mary GrandPré,4.28,4


## Вывод
При учёте книг, собравших больше 50 оценок, самый высокий рейтинг у J.K. Rowling/Mary GrandPré. В подсчёте участвовало 4 книги. 

## Среднее количество обзоров от пользователей 
Учитывать будем только пользователей, которые поставили больше 48 оценок. 

In [14]:
query = """
WITH active_users AS (
    SELECT 
        username,
        COUNT(rating_id) AS ratings_count
    FROM 
        ratings
    GROUP BY 
        username
    HAVING 
        COUNT(rating_id) > 48
),
user_reviews AS (
    SELECT 
        r.username,
        COUNT(r.review_id) AS reviews_count
    FROM 
        reviews r
    WHERE 
        r.username IN (SELECT username FROM active_users)
    GROUP BY 
        r.username
)
SELECT 
    ROUND(AVG(reviews_count), 2) AS avg_reviews_per_active_user
FROM 
    user_reviews;
"""

response = pd.io.sql.read_sql(query, con = engine)
response

,avg_reviews_per_active_user
0,24.0


## Вывод 
Пользователи, оставившие больше 48 оценок, в среднем пишут по 24 отзыва. 

# Выводы 
Собрали необходимую информацию по всем запросам для дальнейшего анализа. 